In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import logging

In [2]:
url='https://www.airlinequality.com/airline-reviews/british-airways/'

In [7]:
# Python script typically indicates that the server understood your request but refuses to authorize it.
# This is a common issue when trying to access web pages programmatically, especially on websites that have measures to prevent or limit web scraping
#User-Agent is set to mimic a request from a Google Chrome browser on Windows 10. This might help in bypassing the basic scraping blocks


from urllib.request import Request, urlopen

url='https://www.airlinequality.com/airline-reviews/british-airways/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

req = Request(url, headers=headers)
response = urlopen(req)
content = response.read()


In [8]:
url_html=bs(content, 'html5')

In [9]:
big_box=url_html.find_all('div', class_="text_content")

In [10]:
len(big_box)

10

In [11]:
big_box[1].get_text(strip=True)

'Not Verified|  Overall not bad. Staff look tired and overworked, trying their best. Weird seats, I was in 21G going, and 21D going home. The buttons to press for the seat recline are so stiff I had to get help on both flights to get them to work. There is another button, also very stiff, to bring the “leg rest” out. This is an awkward thing which doesn’t come up far enough to be of any use at all for your legs, but just obstructs your feet really. Again, I had to get help to activate the very stiff buttons. At one point, the lady next to me was pressing the buttons for me while I tried to recline, sit up, or put the “leg rest” back. We gave up and called the flight attendant, who also had trouble. BA: please give this some maintenance! Food: only one out of three menu choices available by the time it got to me in Premium Ec. It was pasta and was not too bad. A Christmas Turkey Dinner was offered, but why not load more if it is so popular that it is not available 10 minutes into the me

In [12]:
#create an empty list to collect all reviews
reviews  = []

#create an empty list to collect rating stars
stars = []

#create an empty list to collect date
date = []

#create an empty list to collect country the reviewer is from
country = []

In [13]:
# for accessing the all the pages we use the for loop

for i in range(1, 300):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")

    soup = bs(page.content, "html5")

    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)

    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")

    #date
    for item in soup.find_all("time"):
        date.append(item.text)

    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))


Error on page 32
Error on page 33
Error on page 33
Error on page 36
Error on page 36


In [34]:
#check the length of total reviews extracted
len(reviews)
reviews=reviews[:3721]

In [35]:
len(country)
country=country[:3721]

In [36]:
len(date)
date=date[:3721]

In [37]:
#check the length
stars = stars[:3721]

In [38]:
print(len(reviews), len(stars), len(date), len(country))


3721 3721 3721 3721


In [39]:
#create  a dataframe from these collected lists of data

df = pd.DataFrame({"reviews":reviews,"stars": stars, "date":date, "country": country})

In [40]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | Absolutely PATHETIC business...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,23rd December 2023,United States
1,Not Verified | Overall not bad. Staff look ti...,3,21st December 2023,Canada
2,✅ Trip Verified | This was our first flight wi...,6,21st December 2023,Australia
3,✅ Trip Verified | I recently encountered a hig...,10,21st December 2023,United States
4,Not Verified | Beware! BA don't provide any r...,1,20th December 2023,United States


In [41]:
import os

cwd = os.getcwd()
df.to_csv(cwd+ "/BA_reviews.csv")